# Speed Skydiving scoring bulk files processor
Uses the `./data` directory as a data lake.  It finds all valid FlySight files in that directory, from 0 to as many as it finds and fit in memory, and process them all.  Summary of "all files" or "all jumps" at the bottom of the document.

In [41]:
%%capture
!make local

In [42]:
from collections import namedtuple
from copy import deepcopy

from ssscoring.calc import aggregateResults
from ssscoring.calc import convertFlySight2SSScoring
from ssscoring.calc import dropNonSkydiveDataFrom
from ssscoring.calc import getSpeedSkydiveFrom
from ssscoring.calc import isValidJump
from ssscoring.calc import isValidMinimumAltitude
from ssscoring.calc import jumpAnalysisTable
from ssscoring.calc import processAllJumpFiles
from ssscoring.calc import roundedAggregateResults
from ssscoring.calc import totalResultsFrom
from ssscoring.constants import BREAKOFF_ALTITUDE
from ssscoring.constants import FT_IN_M
from ssscoring.constants import PERFORMANCE_WINDOW_LENGTH
from ssscoring.flysight import getAllSpeedJumpFilesFrom
from ssscoring.flysight import validFlySightHeaderIn
from ssscoring.notebook import SPEED_COLORS
from ssscoring.notebook import graphAltitude
from ssscoring.notebook import graphAngle
from ssscoring.notebook import graphJumpResult
from ssscoring.notebook import initializeExtraYRanges
from ssscoring.notebook import initializePlot

import csv
import os
import os.path as path

import bokeh.plotting as bp
import bokeh.models as bm
import ipywidgets as widgets
import pandas as pd

In [43]:
DATA_LAKE_ROOT = './data'

---
## <span style="color: yellow">IMPORTANT</span> - Set the drop zone altitude MSL

Set the value in ft (even though SSScoring uses meters) out of convenience because all DZs in the US, many in Asia and Mexico, and many in Europe express the altitude in feet.

### Some DZs

| Drop zone | Alt (ft) |
|-----------|----------|
| Bay Area Skydiving | 23 |
| Paraclete XP| 304 |
| SkyDance SkyDiving | 100 |
| Skydive Arizona | 1509 |
| Skydive Chicago | 616 |
| Thai Sky Adventures | 21 |

In [44]:
dropZoneAltMSL = 100
ignoreBaseline = False # Set to False to evaluate any baseline tracks present in the data lake.

In [45]:
dropZoneAltMSLMeters = dropZoneAltMSL/FT_IN_M
display(widgets.HTML('<h2>DZ Altitude = <span style = "color: green">%7.2f ft</span> (%7.2f m)<h1>' % (dropZoneAltMSL, dropZoneAltMSLMeters)))

HTML(value='<h2>DZ Altitude = <span style = "color: green"> 100.00 ft</span> (  30.48 m)<h1>')

In [46]:
jumpFiles = getAllSpeedJumpFilesFrom(DATA_LAKE_ROOT)

In [47]:
%%capture warnings
jumpResults = processAllJumpFiles(jumpFiles, altitudeDZMeters = dropZoneAltMSLMeters)

### Air density

**<a href='https://aviex.goflexair.com/blog/density-of-air-air-density' target='_blank'>Pilot's Guide to Air Density</a>** - A useful guide with calculations, lots of information.

![](https://ciurana.eu/personal/air-density-partial-table.png)

---
## Results

In [48]:
aggregate = aggregateResults(jumpResults)
if ignoreBaseline:
    aggregate = aggregate[~aggregate.index.str.contains('baseline')]
sumResults = totalResultsFrom(aggregate)
display(aggregate)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 00-00-00-baseline-10-50-09:v1,493.77,182.556,332.892,431.028,482.184,493.848,24.2,495.108
data 09-07-30:v1,482.27,179.316,325.116,420.444,470.772,477.288,24.5,484.560
data 10-09-20:v1,487.86,184.932,320.904,421.272,475.668,484.128,24.4,487.044
data 11-00-34:v1,476.31,185.040,332.820,429.876,481.428,315.252,24.9,481.464
data 12-24-08:v1,473.95,179.892,321.588,412.740,461.808,469.800,24.6,472.608


### Rounded results for training log

In [49]:
roundedResults = roundedAggregateResults(aggregate)

### All jumps

In [50]:
def dumpSimple(table: pd.DataFrame):
    fileName = os.path.join(DATA_LAKE_ROOT, resultRef+'.tsv')
    simpleTable = table.copy()
    simpleTable = simpleTable.drop('speedAngle', axis = 1)
    simpleTable.to_csv(fileName, sep = '\t', index = False)

In [51]:
def displayJumpDataIn(resultsTable: pd.DataFrame):
    table = resultsTable.copy()
    # Experimental
    # For more information on the `interpolate` method and its options, see the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html).
    # Additionally, you can also consider using other libraries like `scipy` which provides more advanced interpolation methods, such as `interp1d` or `griddata`. See the [scipy documentation](https://docs.scipy.org/doc/scipy/reference/interpolate.html) for more information.
    table.vKMh = table.vKMh.apply(round)
    table.hKMh = table.hKMh.apply(round)
    table['altitude (ft)'] = table['altitude (ft)'].apply(round)
    table.netVectorKMh = table.netVectorKMh.apply(round)
    table.index = ['']*len(table)
    display(table)
    # dumpSimple(table)

In [52]:
allJumpsPlot = initializePlot('All jumps in set')
jumpNumber = 0
mixColor = 0
for resultRef in sorted(jumpResults.keys()):
    if ignoreBaseline and 'baseline' in resultRef:
        continue
    jumpResult = jumpResults[resultRef]
    if jumpResult.score > 0.0:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s - score = %.02f km/h</span></h1>' % (jumpResult.color, resultRef, jumpResult.result, jumpResult.score)        
    else:
        validJumpStatus = '<hr><h1><span style="color: %s">%s jump - %s</span></h1>' % (jumpResult.color, resultRef, jumpResult.result)
    display(widgets.HTML(validJumpStatus))

    maxSpeed = jumpResult.maxSpeed
    window = jumpResult.window
    mixColor = (mixColor+1)%len(SPEED_COLORS)
    if jumpResult.score > 0.0:
        display(widgets.HTML('<h3>Max speed = {0:,.0f}; '.format(maxSpeed)+('exit at %d m (%d ft), end scoring window at %d m (%d ft)'%(window.start, 3.2808*window.start, window.end, 3.2808*window.end))))
        if not isValidMinimumAltitude(jumpResult.data.altitudeAGL.max()):
            display(widgets.HTML('<span style="color: yellow"><span style="font-weight: bold">Warning:</span> exit altitude ASL was lower than the minimum scoring altitude according to IPC and USPA.'))
        display(widgets.HTML('</h3>'))
        # TODO: Move this to an in-notebook function or module.
        displayJumpDataIn(jumpResult.table)
        individualPlot = initializePlot(resultRef)
        individualPlot = initializeExtraYRanges(individualPlot, startY = min(jumpResult.data.altitudeAGLFt)-500.0, endY = max(jumpResult.data.altitudeAGLFt)+500.0)
        graphAltitude(individualPlot, jumpResult)
        graphAngle(individualPlot, jumpResult)
        hoverValue = bm.HoverTool(tooltips = [ ('Y-val', '@y{0.00}'), ])
        individualPlot.add_tools(hoverValue)
        graphJumpResult(individualPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[0])
        graphJumpResult(allJumpsPlot,
                        jumpResult,
                        lineColor = SPEED_COLORS[mixColor],
                        legend = '%s - %.2f' % (resultRef, jumpResult.score),
                        showIt = False)

HTML(value='<hr><h1><span style="color: #0f0">data 00-00-00-baseline-10-50-09:v1 jump - 🟢 valid - score = 493.…

HTML(value='<h3>Max speed = 495; exit at 4211 m (13816 ft), end scoring window at 1955 m (6414 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,183,154,49.91,252.04,13385,239
,10.0,333,109,71.91,432.42,12158,350
,15.0,431,62,81.86,547.29,10428,435
,20.0,482,14,88.29,593.79,8325,482
,25.0,494,9,88.91,590.15,6442,494


HTML(value='<hr><h1><span style="color: #0f0">data 09-07-30:v1 jump - 🟢 valid - score = 482.27 km/h</span></h1…

...

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,185,106,60.21,184.55,13369,213
,10.0,333,69,78.36,305.88,12137,340
,15.0,430,42,84.37,371.05,10406,432
,20.0,481,37,85.64,385.35,8353,483
,25.0,315,41,82.62,374.17,6456,318


HTML(value='<hr><h1><span style="color: #0f0">data 12-24-08:v1 jump - 🟢 valid - score = 473.95 km/h</span></h1…

HTML(value='<h3>Max speed = 473; exit at 4133 m (13560 ft), end scoring window at 1877 m (6159 ft)')

HTML(value='</h3>')

,time,vKMh,hKMh,speedAngle,distanceFromExit (m),altitude (ft),netVectorKMh
,5.0,180,105,59.65,174.50,13117,208
,10.0,322,80,76.04,299.85,11935,331
,15.0,413,66,80.96,395.19,10223,418
,20.0,462,55,83.25,451.67,8198,465
,25.0,470,50,83.88,470.28,6220,472


---
## Summary of all speed skydives in the data lake

**<a href='https://intimescoring.com/Results/EventResults?dbid=100&mid=1074&smid=0&mdcid=1273&dseid=0' target='_blank'>InTime Scoring - 2024 USPA Speed Open results</a>**

In [53]:
display(roundedResults)
display(sumResults)
bp.show(allJumpsPlot)

,score,5.0,10.0,15.0,20.0,25.0,finalTime,maxSpeed
data 00-00-00-baseline-10-50-09:v1,494,183,333,431,482,494,24.2,495
data 09-07-30:v1,482,179,325,420,471,477,24.5,485
data 10-09-20:v1,488,185,321,421,476,484,24.4,487
data 11-00-34:v1,476,185,333,430,481,315,24.9,481
data 12-24-08:v1,474,180,322,413,462,470,24.6,473


,totalSpeed,meanSpeed,maxScore
totalSpeed,2414.16,482.832,493.77
